# Работа с базами данных (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа с базами данных"
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.sqlitetutorial.net/sqlite-index/
* https://docs.python.org/3/library/sqlite3.html#sqlite3.IntegrityError
* https://www.sqlitetutorial.net/sqlite-alter-table/
* https://www.sqlitetutorial.net/sqlite-create-view/
* https://habr.com/ru/post/664000/
* https://learnsql.com/blog/what-is-common-table-expression/


## Задачи для совместного разбора

In [1]:
import pandas as pd
import sqlite3

# данные
students = pd.DataFrame(
    [
        ("Сотников Евгений Янович", 1),
        ("Степанова Виктория Константиновна", 1),
        ("Горелова Вероника Яновна", 2),
        ("Гришин Иван Романович", 3),
    ],
    columns=["name", "group_id"],
)
groups = list(zip([1, 2, 3], ["ПМ20-1", "ПМ20-2", "ПМ20-3"]))

con = sqlite3.connect("demo.sqlite")
con.execute("PRAGMA foreign_keys = 1")
cur = con.cursor()

# создаем таблицы
sql = """
DROP TABLE IF EXISTS StudentGroup;
DROP TABLE IF EXISTS Student;
CREATE TABLE StudentGroup (
    id int PRIMARY KEY,
    name varchar
);

CREATE TABLE Student(
    name VARCHAR PRIMARY KEY,
    group_id INT,
    FOREIGN KEY (group_id) REFERENCES StudentGroup(id)
);
"""
cur.executescript(sql)
con.commit()

# добавляем записи
sql = """
INSERT INTO StudentGroup(id, name) VALUES (?, ?)
"""
cur.executemany(sql, groups)
con.commit()

students.to_sql("Student", con, if_exists="append", index=False)

1\. Добавить столбец Age со значением по умолчанию. Добавить запись к таблицу

In [2]:
sql = '''
ALTER TABLE Student
    ADD COLUMN Age INT DEFAULT 18
'''
cur.execute(sql)
con.commit()

In [3]:
pd.read_sql_query('SELECT * FROM Student', con)

,name,group_id,Age
0,Сотников Евгений Янович,1,18
1,Степанова Виктория Константиновна,1,18
2,Горелова Вероника Яновна,2,18
3,Гришин Иван Романович,3,18


In [6]:
sql = '''
INSERT INTO Student(name,group_id,Age)
    VALUES ('Солончукова Полина Станиславовна', 2, 19)
'''
try:
    cur.execute(sql)
except sqlite3.OperationalError as e:
    print('Произошла ошибка:', e)
else:
    print('Ошибки не произошло')
    con.commit()
con.commit()

Ошибки не произошло


2\. Занумеруйте студентов в рамках каждой группы.

In [17]:
sql = '''
SELECT name,
       group_id,
       Age,
       ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY Age DESC) as rid
FROM Student
'''
pd.read_sql_query(sql,con)

,name,group_id,Age,rid
0,Сотников Евгений Янович,1,18,1
1,Степанова Виктория Константиновна,1,18,2
2,Солончукова Полина Станиславовна,2,19,1
3,Горелова Вероника Яновна,2,18,2
4,Гришин Иван Романович,3,18,1


3\. Выведите уникальные номера студентов

In [19]:
sql = '''
SELECT DISTINCT rid
FROM (
    SELECT name,
           group_id,
           Age,
           ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY Age DESC) as rid
    FROM Student
)
'''
pd.read_sql_query(sql,con)

,rid
0,1
1,2


In [20]:
sql = '''
SELECT DISTINCT ROW_NUMBER() OVER(PARTITION BY group_id) as rid
FROM Student
'''
pd.read_sql_query(sql,con)

,rid
0,1
1,2


In [21]:
# представление
sql = '''
CREATE VIEW StudentWithNumberView AS 
    SELECT name,
           group_id,
           Age,
           ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY Age DESC) as rid
    FROM Student
'''
cur.execute(sql)
con.commit()

In [22]:
sql = '''
SELECT DISTINCT rid
FROM StudentWithNumberView
'''
pd.read_sql_query(sql,con)

,rid
0,1
1,2


In [23]:
# обобщенное табличное выражение
sql = '''
WITH StudentWithNumberCTE AS (
    SELECT name,
           group_id,
           Age,
           ROW_NUMBER() OVER(PARTITION BY group_id ORDER BY Age DESC) as rid
    FROM Student
)

SELECT DISTINCT rid
FROM StudentWithNumberView
'''
pd.read_sql_query(sql,con)

,rid
0,1
1,2


## Лабораторная работа 4

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

In [2]:
import pandas as pd
import sqlite3

__Для начала работы подключитесь к БД `recipes.db` и создайте объект-курсор.__

In [464]:
#con = sqlite3.connect('recipes.db')
#cur = con.cursor()

In [567]:
sqlite3.register_adapter(bool, int)
sqlite3.register_converter("BOOLEAN", lambda x: bool(int(x)))
con = sqlite3.connect('recipes.db', detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()

In [569]:
#cur.execute('ALTER TABLE Review DROP COLUMN toxic')

<p class="task" id="1"></p>

1\. Создайте уникальный индекс для таблицы `Review` для обеспечения уникальности сочетания значений в полях `user_id` и `recipe_id`. 

In [268]:
pd.read_sql_query('SELECT * FROM Review',con).head()

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [571]:
#cur.execute('DROP INDEX user_recipe_id;')
#con.commit()

In [572]:
sql = '''
CREATE UNIQUE INDEX user_recipe_id 
ON Review (user_id, recipe_id);
'''
cur.execute(sql)
con.commit()

In [573]:
sql = '''PRAGMA index_list('Review');'''
pd.read_sql_query(sql, con)

,seq,name,unique,origin,partial
0,0,user_recipe_id,1,c,0
1,1,sqlite_autoindex_Review_1,1,pk,0


In [574]:
sql = '''PRAGMA index_info('user_recipe_id');'''
pd.read_sql_query(sql, con)

,seqno,cid,name
0,0,1,user_id
1,1,2,recipe_id


<p class="task" id="2"></p>

2\. Напишите функцию `add_review(review_id, user_id, recipe_id, date, rating, review)`, которая добавляет запись в таблицу `Review`. В случае успешного добавления функция должна вернуть значение 0. В случае нарушения ограничения целостности функция должна вернуть значение 1. В случае любых других ошибок функция должна вернуть значение 2. Продемонстрируйте работу функции, попытавшись добавить одну и ту же запись дважды в двух ячейках подряд.

Для решения задачи воспользуйтесь механизмом try - except и обработайте соответствующее исключение.

In [575]:
def add_review(review_id, user_id, recipe_id, date, rating, review):
    sql = '''
    INSERT INTO Review(id, user_id, recipe_id, date, rating, review)
    VALUES (?,?,?,?,?,?)
    '''
    try:
        cur.execute(sql, [review_id, user_id, recipe_id, date, rating, review])
        con.commit()
        return 0
    except sqlite3.Error as e:
        if isinstance(e, sqlite3.IntegrityError):
            return 1
        else:
            return 2

In [1]:
def add_review_2(review_id, user_id, recipe_id, date, rating, review):
    sql = '''
    INSERT INTO Review(id, user_id, recipe_id, date, rating, review)
    VALUES (?,?,?,?,?,?)
    '''
    try:
        cur.execute(sql, [review_id, user_id, recipe_id, date, rating, review])
        con.commit()
        return 0
    except BaseException as e:
        if isinstance(e, sqlite3.IntegrityError):
            return 1
        else:
            return 2

In [576]:
add_review(2,1,1,'2003-05-01',4,'descripion') # несуществующее сочетание

0

In [577]:
sql = '''
SELECT * FROM Review WHERE id = 2
'''
pd.read_sql_query(sql, con)

,id,user_id,recipe_id,date,rating,review
0,2,1,1,2003-05-01,4,descripion


In [578]:
add_review(3,1,1,'2020-05-01',777,'inf') # существующее сочетание

1

In [579]:
sql = '''
SELECT * FROM Review WHERE id = 3
'''
pd.read_sql_query(sql, con)

,id,user_id,recipe_id,date,rating,review


In [580]:
sql = ''' 
DELETE FROM Review WHERE id = 2;
'''
cur.execute(sql)
con.commit()

In [581]:
sql = '''
SELECT * FROM Review WHERE id = 2;
'''
pd.read_sql_query(sql, con)

,id,user_id,recipe_id,date,rating,review


In [582]:
add_review(3,1,1,'2020-05-01',[1,2,3],'inf')

2

In [583]:
sql = '''
SELECT * FROM Review WHERE id = 3
'''
pd.read_sql_query(sql, con)

,id,user_id,recipe_id,date,rating,review


<p class="task" id="3"></p>

3\. _Измените_ таблицу Review, добавив в нее поле `toxic` булева типа. 

In [667]:
#cur.execute('ALTER TABLE Review DROP COLUMN toxic')

In [668]:
sql = '''
ALTER TABLE Review
    ADD COLUMN toxic BOOLEAN
'''
cur.execute(sql)
con.commit()

In [669]:
pd.read_sql_query('SELECT * FROM Review',con).head()

,id,user_id,recipe_id,date,rating,review,toxic
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,None
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,None
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",None
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,None
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,None


In [670]:
sql = '''
INSERT INTO Review(id, user_id, recipe_id, date, rating, review, toxic)
VALUES (2,1,1,'2003-05-01',4,'descripion',True);

INSERT INTO Review(id, user_id, recipe_id, date, rating, review, toxic)
VALUES (3,3,3,'2003-05-01',4,'descripion',1)
'''
cur.executescript(sql)
con.commit()

In [671]:
pd.read_sql_query('SELECT * FROM Review',con)['toxic'].tail()

126693    None
126694    None
126695    None
126696    True
126697    True
Name: toxic, dtype: object

In [672]:
sql = ''' 
DELETE FROM Review WHERE (id = 2) OR (id = 3);
'''
cur.execute(sql)
con.commit()

In [673]:
pd.read_sql_query('SELECT * FROM Review',con)['toxic'].tail()

126691    None
126692    None
126693    None
126694    None
126695    None
Name: toxic, dtype: object

<p class="task" id="4"></p>

4\. Вам дан классификатор `clf`, который классифицирует тексты отзывов как токсичные (`True`) и не токсичные (`False`).
Напишите функцию `classify_reviews`, которая итеративно получает пакет (батч) `batch_size` строк из таблицы Reviews, у которых не проставлено значение в столбце `toxic`, делает для них прогноз при помощи модели `clf` и обновляет соответствующие строки в БД. Данная процедура выполняется до тех пор, пока в БД есть строки, для которых требуется получить прогноз.

Продемонстрируйте результат, выведя на экран количество токсичных и не токсичных отзывов в таблице.

In [674]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy="uniform").fit(None, [True, False])

In [679]:
def classify_reviews(batch_size=10000):
    sql = f'''
    SELECT id 
    FROM Review 
    WHERE toxic is Null
    LIMIT {batch_size}
    '''
    df_clf = pd.read_sql_query(sql, con)
    pred = list(map(int,clf.predict(df_clf)))

    while not df_clf.empty:
        sql_update_query = '''UPDATE Review SET toxic = (?) WHERE id = (?)'''
        cur.executemany(sql_update_query, zip(pred, pd.to_numeric(df_clf['id'], downcast='integer')))
        con.commit()
        df_clf = pd.read_sql_query(sql, con)
        pred = list(map(int,clf.predict(df_clf)))

    return pd.read_sql_query('SELECT * FROM Review', con)

In [680]:
df_4 = classify_reviews()

In [681]:
df_4.head()

,id,user_id,recipe_id,date,rating,review,toxic
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,True
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,False
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",False
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,False
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,False


In [678]:
df_4['toxic'].unique()

array([ True, False])

In [694]:
df_4['toxic'].value_counts() 

False    63369
True     63327
Name: toxic, dtype: int64

<p class="task" id="5"></p>

5\. Создайте представление `RecipeWithYear`, в котором добавлен дополнительный столбец `year`, содержащий год даты из столбца `submitted`. Сделайте выборку из этого представления и выведите на экран количество рецептов с разбивкой по годам.

In [717]:
sql = '''
CREATE VIEW RecipeWithYear AS 
   SELECT *, strftime('%Y', submitted) as year
   FROM Recipe
'''
cur.execute(sql)
con.commit()

In [723]:
sql = '''
SELECT year, count(id) as count
FROM RecipeWithYear
GROUP BY year
'''
pd.read_sql_query(sql, con)

,year,count
0,1999,275
1,2000,104
2,2001,589
3,2002,2644
4,2003,2334
5,2004,2153
6,2005,3130
7,2006,3473
8,2007,4429
9,2008,4029


In [716]:
#cur.execute('drop view RecipeWithYear')

<p class="task" id="6"></p>

6\. Напишите запрос на языке SQL, который возвращает все строки из таблицы `Recipe` с дополнительным столбцом, содержащем номер рецепта. Рецепты нумеруются целыми числами, начиная с 1, в __рамках каждого года__ в порядке их добавления в БД (столбец `submitted`). Получите результат в виде `pd.DataFrame`. Посчитайте и выведите на экран количество строк полученного `pd.DataFrame`, для которых сгенерированный номер кратен 50.

In [725]:
sql = '''
SELECT *, ROW_NUMBER() OVER(PARTITION BY year ORDER BY submitted) as rid 
FROM RecipeWithYear
'''
df6 = pd.read_sql_query(sql, con)
df6

,id,name,minutes,submitted,description,n_ingredients,year,rid
0,203,chinese plum sauce,115,1999-08-06,chinese plum sauce serve this with egg rolls ...,12.0,1999,1
1,653,b c cherry and raspberry preserves,215,1999-08-09,None,4.0,1999,2
2,360,baked zucchini frittatas,67,1999-08-09,None,NaN,1999,3
3,658,dried fruit roll ups,1495,1999-08-09,fruit roll-ups,4.0,1999,4
4,1144,steak tomato basil pasta,0,1999-08-09,None,11.0,1999,5
...,...,...,...,...,...,...,...,...
29995,536547,cauliflower ceviche,45,2018-07-30,a healthy ceviche - a perfect appetizer for pa...,8.0,2018,20
29996,536610,miracle home made puff pastry,35,2018-07-31,puff pastry that you can make in minutes? at h...,NaN,2018,21
29997,536729,creole watermelon feta salad,10,2018-08-11,spicy watermelon salad. from tony chachere's s...,NaN,2018,22
29998,536728,gluten free vegemite,2,2018-08-11,gluten free vegemite-like stuff.,3.0,2018,23


In [731]:
df6[df6['rid']%50==0]

,id,name,minutes,submitted,description,n_ingredients,year,rid
49,246,lee s hot crab dip,45,1999-09-01,lee's hot crab dip,10.0,1999,50
99,2891,potluck pasta salad,0,1999-09-27,originally from,12.0,1999,100
149,3441,30 minute smoked sausage and corn chowder,30,1999-10-18,"i love corn chowder, have a pot on now! recipe...",8.0,1999,150
199,4279,never fail pie crust ii,0,1999-11-10,None,6.0,1999,200
249,4931,pickling corned beef,17340,1999-12-01,None,7.0,1999,250
...,...,...,...,...,...,...,...,...
29591,504794,green peas pork chow yoke,25,2013-07-28,"from desmoines daily register; september 8, 19...",15.0,2013,350
29641,506648,moroccan carrot and chickpea tagine,50,2013-09-05,"this spicy dish uses ras el hanout, one of my ...",16.0,2013,400
29691,508912,delicious vinaigrette,15,2013-11-04,this recipe was was developed over many hundre...,NaN,2013,450
29781,513886,herbamare substitute,10,2014-03-02,herbamare is usually expensive and hard to fin...,12.0,2014,50


In [732]:
df6[df6['rid']%50==0].shape[0]

589

<p class="task" id="7"></p>

7\. Используя обобщенное табличное выражение и решение задачи 6, напишите запрос на языке SQL, который вернет количество строк, для которых сгенерированный номер кратен 50. Выполните запрос и выведите количество таких строк на экран.

In [735]:
sql = '''
WITH RecipeWithRid AS (
    SELECT *, ROW_NUMBER() OVER(PARTITION BY year ORDER BY submitted) as rid 
    FROM RecipeWithYear
)

SELECT COUNT(id)
FROM RecipeWithRid
WHERE rid % 50 = 0
'''
cur.execute(sql).fetchall()

[(589,)]